In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# General Imports

In [2]:
!pip install GPUtil                   # to check GPU usage
!pip install -q bs4                   # to clean the lyrics
!pip install -q datasets
!pip install transformers             # for training
# Update Transformers from the source if you get errors:
# !pip install git + https://github.com/huggingface/transformers
!pip install tf-models-official       # for optimization
!pip install pyyaml h5py              # to save the models

"""
If you have ongoing issues with the transformers library, Also consider installing from source: 
git clone https://github.com/huggingface/transformers.git 
cd transformers (in Dropbox/C/25_Studium_Master/TUM_RCI/2021_WS/NLP/3-Exercises-Workspace/transformers)
pip install -e
Also update git
"""

  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=f8160e4cbb85f4dad357b46839aac6a322a247c59e1d333728680adf38886196
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
     |████████████████████████████████| 312 kB 4.2 MB/s 
     |████████████████████████████████| 134 kB 47.0 MB/s 
     |████████████████████████████████| 212 kB 53.6 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 54.9 MB/s 
     |████████████████████████████████| 127 kB 59.5 MB/s 
     |████████████████████████████████| 271 kB 60.4 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 61.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 require

'\nIf you have ongoing issues with the transformers library, Also consider installing from source: \ngit clone https://github.com/huggingface/transformers.git \ncd transformers (in Dropbox/C/25_Studium_Master/TUM_RCI/2021_WS/NLP/3-Exercises-Workspace/transformers)\npip install -e\nAlso update git\n'

In [3]:
import tensorflow as tf
from tensorflow import keras
from transformers import AutoConfig, AutoTokenizer, TFAutoModelForCausalLM, default_data_collator, pipeline 
# DefaultDataCollator: for the error https://github.com/huggingface/transformers/pull/5015
from official.nlp import optimization
from datasets import Dataset      # https://huggingface.co/docs/datasets/
from bs4 import BeautifulSoup     # easy webscraping
import requests                   # for sending http requests
import re                         # regular expressions, excellent for string searching/replacing, 
# extremely powerful for manual NLP, check out the documentation at https://docs.python.org/3/library/re.html
import os                         # accessing file paths
# os.environ["CUDA_VISIBLE_DEVICES"]="-1" to deactive GPU
import json
import random
from collections import Counter
import math
import GPUtil
from numba import cuda 

model_name = "gpt2"               # selected pretrained language model from https://huggingface.co/models

# "gtp3" is huge, needed several GPUs for traning, didnt fit in to Colab Pro Machine. 
# I tried distilgpt2 on my machine. But even then I ran in to problems with OOM 
# errors often before switching to Google Colab Pro (10euro/month) for efficiency
# as I only have a GeForce GTX M 860 with 4 GB memory. I didn't try distilgpt2
# here again as I direct could use gpt2.

In [4]:
#GPUtil.showUtilization()

# Downloading Lyrics

In [5]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: Changed the directories to save genres separately 

# I used the my own machine here to save time during the downloading of lyrics, 
# That is why the above download paths are for my own PC instead of drive. 
# This cell and the above 6 needs to be ran own your own machine as your CPU is 
# Faster then Google Colabs GPU for downloads. Then the files need to be copied
# to drive afterwards and the drive has to be mounted as it's done at the start
# of the notebook 

# Commented out to make the restarting and runnning easier as this is needed once 

# USE THE LOCAL ONE!!!!

# helper functions
# insert links to lyricsfreak artist pages
"""
artist_links = [
    "https://www.lyricsfreak.com/b/blues+brothers/"
]
    

# helper functions
def get_links(artist_link):
    page = requests.get(artist_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    a = soup.findAll("a", {"class": "lf-link lf-link--primary"})
    links = [i["href"] for i in a]
    print(str(len(links)) + " lyrics found at " + artist_link)
    return(links)

def get_lyrics(links):
    lyrics = {}
    for l in links:
        page = requests.get("https://www.lyricsfreak.com" + str(l))
        soup = BeautifulSoup(page.content, 'html.parser')
        title = re.sub("\+", "_", l.split("/")[-1].split("_")[0])
        lyrics[title] = soup.find(id = "content")
    return(lyrics)

# some strings in the lyrics that will be removed
beg = re.compile('<div class.*')
end = re.compile('</div>')
lin = re.compile('<br/>')
cho = re.compile('chorus|verse|intro', flags = re.IGNORECASE)
cr = re.compile('\r')
patterns = [beg, end, lin, cho, cr]

def clean_text(patterns, text):
    text = str(text)
    for pattern in patterns:
        text = re.sub(pattern, "", text)
    return(text.strip())
        
lyrics_dict = {}

for artist_link in artist_links:
    
    lyrics_links = [link for link in (get_links(artist_link))]
    lyrics = get_lyrics(lyrics_links)
    lyrics = {title: clean_text(patterns,lyric) for title, lyric in lyrics.items()}
    
    lyrics_dict[re.sub("\+", "_", artist_link.split("/")[-2])] = lyrics

with open(save_path + "one_artist.json", "w", encoding = "utf8") as f:
    
    json.dump(lyrics_dict, f)

"""

'\nartist_links = [\n    "https://www.lyricsfreak.com/b/blues+brothers/"\n]\n    \n\n# helper functions\ndef get_links(artist_link):\n    page = requests.get(artist_link)\n    soup = BeautifulSoup(page.content, \'html.parser\')\n    a = soup.findAll("a", {"class": "lf-link lf-link--primary"})\n    links = [i["href"] for i in a]\n    print(str(len(links)) + " lyrics found at " + artist_link)\n    return(links)\n\ndef get_lyrics(links):\n    lyrics = {}\n    for l in links:\n        page = requests.get("https://www.lyricsfreak.com" + str(l))\n        soup = BeautifulSoup(page.content, \'html.parser\')\n        title = re.sub("\\+", "_", l.split("/")[-1].split("_")[0])\n        lyrics[title] = soup.find(id = "content")\n    return(lyrics)\n\n# some strings in the lyrics that will be removed\nbeg = re.compile(\'<div class.*\')\nend = re.compile(\'</div>\')\nlin = re.compile(\'<br/>\')\ncho = re.compile(\'chorus|verse|intro\', flags = re.IGNORECASE)\ncr = re.compile(\'\r\')\npatterns = [beg

# Generator <a class="anchor" id="Generator"></a>

In [6]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: Changed the directories to save genres separately 

# loading saved lyrics files

with open("/content/drive/MyDrive/EchoCanyon/lyrics/my_artists.json", "r", encoding = "utf8") as f:

    lyrics_dict = {}
    lyrics_dict = json.load(f)

In [7]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# reformatting lyrics

lyrics = {"text":[], "artist":[]}
for artist, titles in lyrics_dict.items():
    
    for title, text in titles.items():
        
        lyrics["artist"].append(artist)
        lyrics["text"].append(text)

In [8]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# AutoTokenizer will find and load the right tokenizer for all common models in the huggingface library
tokenizer = AutoTokenizer.from_pretrained(model_name)

i = random.randint(0, len(lyrics["text"]))
example_text = lyrics["text"][i][0:200]
example_text_tokenized = tokenizer(example_text)
print([tokenizer.decode(token) for token in example_text_tokenized["input_ids"]])

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

['Come', ' on', ',', ' oh', ' bab', 'a', ' don', "'t", ' you', ' want', ' to', ' go', '\n', 'Oh', ' come', ' on', ',', ' oh', ' bab', 'a', ' don', "'t", ' you', ' want', ' to', ' go', '\n', 'Back', ' to', ' that', ' same', ' old', ' place', '\n', 'Sweet', ' home', ' Chicago', '\n', '\n', 'Come', ' on', ',', ' bab', 'a', ' don', "'t", ' you', ' want', ' to', ' go', '\n', 'H', 'ida', '-', 'hey', ',', ' bab', 'a', ' don', "'t", ' you', ' want', ' to', ' go', '\n', 'Ba']


In [9]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# transforming lyrics into dataset format

dataset = Dataset.from_dict(lyrics)
print(dataset)

Dataset({
    features: ['text', 'artist'],
    num_rows: 66
})


In [10]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# tokenizing before chunking, to ensure equal lengths

def pretokenization(text, tokenizer):
    tokens = tokenizer(text, truncation = False, add_special_tokens = False, return_attention_mask = False)
    return {"tokens":[tokenizer.decode(token) for token in tokens["input_ids"]]}

tokenized_dataset = dataset.map(lambda x: pretokenization(x["text"],tokenizer), remove_columns = ["text"])
print(tokenized_dataset)
i = random.randint(0, len(dataset["text"]))
print(tokenized_dataset["tokens"][i][0:20])

0ex [00:00, ?ex/s]

Dataset({
    features: ['artist', 'tokens'],
    num_rows: 66
})
['El', 'wood', ' :', '\n', '"', 'Good', ' evening', ' everyone', ' and', ' welcome', ' to', ' the', ' United', ' States', ' of', ' America', '.', ' And', ' indeed', ' we']


In [11]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# chunking the lyrics into parts of equal length
# greater chunks allow longer-range dependencies but also increase computation 
chunk_size = 25  # increases computation quadraticly, basically
# depens on how many close words to consider during training over the current word 
#

def chunking(examples, chunk_size):
    chunks = []
    artists = []
    for tokens, artist in zip(examples["tokens"], examples["artist"]):

        for i in range(0, len(tokens)-chunk_size, chunk_size):
            
            chunks.append(tokens[i:i+chunk_size])
            artists.append(artist)
    
    return {'tokens': chunks, "artist": artists}

chunked_dataset = tokenized_dataset.map(lambda x: chunking(x, chunk_size), batched = True, 
                                        remove_columns = tokenized_dataset.column_names)

# check if all chunks are of equal length
print(max(len(ids) for ids in chunked_dataset["tokens"]))
print(min(len(ids) for ids in chunked_dataset["tokens"]))

print(chunked_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

25
25
Dataset({
    features: ['artist', 'tokens'],
    num_rows: 847
})


In [12]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# stratifying by artist

n = Counter(chunked_dataset["artist"]).most_common()[-1][1]  # from library->Collections

def stratifying(examples, n):  # to have same amount of lines from both artists
    
    chunks = []
    artists = []
    
    for artist in set(examples["artist"]):
        
        artist_chunks = [chunk for a, chunk in zip(examples["artist"], examples["tokens"]) if a == artist]
        artist_chunks = random.sample(artist_chunks, n)
        
        for chunk in artist_chunks:
            
            chunks.append(chunk)
            artists.append(artist)
        
        
    return {'tokens': chunks, "artist": artists}
    
stratified_dataset = chunked_dataset.map(lambda x: stratifying(x, n), batched = True, batch_size = None,
                                        remove_columns = tokenized_dataset.column_names)

print(stratified_dataset)
print(Counter(stratified_dataset["artist"]).most_common())

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['artist', 'tokens'],
    num_rows: 847
})
[('blues_brothers', 847)]


In [13]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# running tokenization again, to retrieve input_ids, labels and the attention_masks

def tokenization(tokens):
    tokenized = tokenizer(tokens)
    return  {"input_ids": tf.squeeze(tokenized["input_ids"]),
             # duplicating the inputs for our labels
             # "the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually"
             "labels": tf.squeeze(tokenized["input_ids"]), 
             "attention_mask": tf.squeeze(tokenized["attention_mask"])}

final_dataset = stratified_dataset.map(lambda x:tokenization(x["tokens"]))
print(final_dataset)

0ex [00:00, ?ex/s]

Dataset({
    features: ['artist', 'tokens', 'input_ids', 'labels', 'attention_mask'],
    num_rows: 847
})


In [14]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# shuffling, batching, splitting  # needed when using tf and it needs to be transfored to a Tensorflow Dataset
from transformers import DefaultDataCollator

batch_size = 2
n_batches = round(final_dataset.num_rows/batch_size)
n_eval_batches = round(0.1*n_batches)
n_train_batches = n_batches - n_eval_batches


# transforming to a tensorflow dataset
data_collator = DefaultDataCollator(return_tensors= "tf")

tf_dataset = final_dataset.to_tf_dataset(columns=["attention_mask", "input_ids", "labels"],
                                                                     shuffle = True, 
                                                                     batch_size = batch_size,
                                                                     collate_fn = data_collator)

# !!!!! Important: setting the seed does not prevent different shuffling at each epoch, 
# !!!!! set reshuffle_each_iteration=False if used


tf_train_dataset = tf_dataset.take(n_train_batches)
tf_eval_dataset = tf_dataset.skip(n_train_batches).take(n_eval_batches)

print(tf_train_dataset)

<TakeDataset element_spec={'labels': TensorSpec(shape=(2, None), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(2, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(2, None), dtype=tf.int64, name=None)}>


In [15]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# inspecting the configuration of the pretrained model
config = AutoConfig.from_pretrained(model_name)

# for more elaboration of all the configuration details check here:
# https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2Config
# print(config)

# "n_ctx": 1024, sequence dim
# "n_embd": 768, embedding dim
# "n_head": 12, num of attetion heads

In [16]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# loading the model including its causal language modeling head

model = TFAutoModelForCausalLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [17]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

# testing the pretrained model as is

generator = pipeline('text-generation',model=model, tokenizer=tokenizer) 

# for more detailed tuning you have to implement differently
# for that the tokenization works
# for different amount of randomness etc.


#prompt = "Peace sells but who is buying"

#print(generator(prompt)[0]["generated_text"])
# so you can see it wasnt trained on lyrics

In [18]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: Hyperparameter Tuning see next section

# initializing the optimizer and defining our learning rate schedule

epochs = 50
num_train_steps = epochs * n_train_batches
warmup_ratio = 0.1
num_warmup_steps = int(warmup_ratio*num_train_steps)

init_lr = 1e-3

optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

# compiling the model
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [19]:
history = model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    epochs = epochs)
    

Epoch 1/50
382/382 [==============================] - 55s 76ms/step - loss: 3.4806 - val_loss: 2.5937
Epoch 2/50
382/382 [==============================] - 25s 67ms/step - loss: 2.8574 - val_loss: 2.0768
Epoch 3/50
382/382 [==============================] - 26s 67ms/step - loss: 2.4835 - val_loss: 1.6762
Epoch 4/50
382/382 [==============================] - 26s 67ms/step - loss: 2.2653 - val_loss: 1.4981
Epoch 5/50
382/382 [==============================] - 25s 67ms/step - loss: 2.0088 - val_loss: 1.4662
Epoch 6/50
382/382 [==============================] - 27s 71ms/step - loss: 1.5570 - val_loss: 0.8916
Epoch 7/50
382/382 [==============================] - 25s 66ms/step - loss: 1.1797 - val_loss: 0.7616
Epoch 8/50
382/382 [==============================] - 26s 67ms/step - loss: 0.9416 - val_loss: 0.5459
Epoch 9/50
382/382 [==============================] - 25s 66ms/step - loss: 0.7620 - val_loss: 0.5026
Epoch 10/50
382/382 [==============================] - 25s 66ms/step - loss: 0.652

In [20]:
#GPUtil.showUtilization()

## Test

In [21]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

"""prompt = "What did the chicken say afer crossing the road"
n_generations = 5 # runs 

lyrics = prompt

for i in range(n_generations):
    
    lyrics += generator(lyrics[-100:], return_full_text = False)[0]["generated_text"]
    
print(lyrics)

#eos enf of sentence is used a  pad token"""

'prompt = "What did the chicken say afer crossing the road"\nn_generations = 5 # runs \n\nlyrics = prompt\n\nfor i in range(n_generations):\n    \n    lyrics += generator(lyrics[-100:], return_full_text = False)[0]["generated_text"]\n    \nprint(lyrics)\n\n#eos enf of sentence is used a  pad token'

# Hyperparameter Tuning

In [22]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

## Imports

%load_ext tensorboard

import os
# to use or not to use GPU
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import json
import random
import re
from collections import Counter
import tensorflow as tf
from tensorflow.keras import layers
from tensorboard.plugins.hparams import api as hp  # for hparam vis
from official.nlp import optimization 
import shutil
import numpy as np
from tensorboard import notebook
from sklearn.utils import class_weight
from sklearn.metrics import precision_recall_fscore_support
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n## Imports\n\n%load_ext tensorboard\n\nimport os\n# to use or not to use GPU\nos.environ["CUDA_VISIBLE_DEVICES"]="-1"\n\nimport json\nimport random\nimport re\nfrom collections import Counter\nimport tensorflow as tf\nfrom tensorflow.keras import layers\nfrom tensorboard.plugins.hparams import api as hp  # for hparam vis\nfrom official.nlp import optimization \nimport shutil\nimport numpy as np\nfrom tensorboard import notebook\nfrom sklearn.utils import class_weight\nfrom sklearn.metrics import precision_recall_fscore_support\n'

In [23]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: Paths

# path to the data
save_path = "/content/drive/MyDrive/EchoCanyon/lyrics/"


# setting up logging directory
log_dir = '/content/drive/MyDrive/EchoCanyon/log_dir/'

try:
    shutil.rmtree(log_dir) # clearing logging directory <------------------
except:
    pass
    
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    """

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: Paths\n\n# path to the data\nsave_path = "/content/drive/MyDrive/EchoCanyon/lyrics/"\n\n\n# setting up logging directory\nlog_dir = \'/content/drive/MyDrive/EchoCanyon/log_dir/\'\n\ntry:\n    shutil.rmtree(log_dir) # clearing logging directory <------------------\nexcept:\n    pass\n    \nif not os.path.exists(log_dir):\n    os.makedirs(log_dir)\n    '

In [24]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# loading your own lyrics file or the one available on moodle
with open(save_path + "pop_lyrics.json", "r", encoding = "utf8") as f:
    
    lyrics_dict = json.load(f)
    
# sampling, if there are more than 5 labels
n_artists = min(5, len(lyrics_dict.keys()))
random.seed(42)
artist_sample = random.sample(list(lyrics_dict.keys()), n_artists)
lyrics_dict = {artist:lyrics for artist, lyrics in lyrics_dict.items() if artist in artist_sample}
print([key for key in lyrics_dict.keys()])
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# loading your own lyrics file or the one available on moodle\nwith open(save_path + "pop_lyrics.json", "r", encoding = "utf8") as f:\n    \n    lyrics_dict = json.load(f)\n    \n# sampling, if there are more than 5 labels\nn_artists = min(5, len(lyrics_dict.keys()))\nrandom.seed(42)\nartist_sample = random.sample(list(lyrics_dict.keys()), n_artists)\nlyrics_dict = {artist:lyrics for artist, lyrics in lyrics_dict.items() if artist in artist_sample}\nprint([key for key in lyrics_dict.keys()])\n'

In [25]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# extracting lines and artists
lines = []
artists = []

for artist, lyrics in lyrics_dict.items():
    
    for title, lyric in lyrics.items():
        
        lyrics_lines = [re.sub("\r", "", line) for line in lyric.split("\n")]
        
        lyrics_lines = [line for line in lyrics_lines if re.search("\w", line) and line != "None"]

        for line in set(lyrics_lines):
            lines.append(line)
            artists.append(artist)

print(len(lines))
print(len(artists))
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# extracting lines and artists\nlines = []\nartists = []\n\nfor artist, lyrics in lyrics_dict.items():\n    \n    for title, lyric in lyrics.items():\n        \n        lyrics_lines = [re.sub("\r", "", line) for line in lyric.split("\n")]\n        \n        lyrics_lines = [line for line in lyrics_lines if re.search("\\w", line) and line != "None"]\n\n        for line in set(lyrics_lines):\n            lines.append(line)\n            artists.append(artist)\n\nprint(len(lines))\nprint(len(artists))\n'

In [26]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# extracting lines and artists
lines = []
artists = []

for artist, lyrics in lyrics_dict.items():
    
    for title, lyric in lyrics.items():
        
        lyrics_lines = [re.sub("\r", "", line) for line in lyric.split("\n")]
        
        lyrics_lines = [line for line in lyrics_lines if re.search("\w", line) and line != "None"]

        for line in set(lyrics_lines):
            lines.append(line)
            artists.append(artist)

print(len(lines))
print(len(artists))
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# extracting lines and artists\nlines = []\nartists = []\n\nfor artist, lyrics in lyrics_dict.items():\n    \n    for title, lyric in lyrics.items():\n        \n        lyrics_lines = [re.sub("\r", "", line) for line in lyric.split("\n")]\n        \n        lyrics_lines = [line for line in lyrics_lines if re.search("\\w", line) and line != "None"]\n\n        for line in set(lyrics_lines):\n            lines.append(line)\n            artists.append(artist)\n\nprint(len(lines))\nprint(len(artists))\n'

In [27]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# Selecting a smaller subset for faster training demonstration
n_lines = 27983

sample = random.sample(range(0,len(lines)), n_lines)
lines = [line for i,line in enumerate(lines) if i in sample]
artists = [artist for i,artist in enumerate(artists) if i in sample]
Counter(artists)
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# Selecting a smaller subset for faster training demonstration\nn_lines = 27983\n\nsample = random.sample(range(0,len(lines)), n_lines)\nlines = [line for i,line in enumerate(lines) if i in sample]\nartists = [artist for i,artist in enumerate(artists) if i in sample]\nCounter(artists)\n'

In [28]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# transforming to a tensorflow dataset

# inputs
inputs = tf.data.Dataset.from_tensor_slices(lines) #.repeat() #https://stackoverflow.com/questions/49531286/cannot-batch-tensors-with-different-shapes-in-component-0-with-tf-data-dataset

input_vectorizer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    split = "whitespace",
    output_mode='int',
    max_tokens = 2000,
    output_sequence_length = 10)

input_vectorizer.adapt(inputs)
inputs = inputs.map(input_vectorizer)

# target dict
artist_dict = {artist: i for i, artist in enumerate(list(set(artists)))}
artist_dict_rev = {i: artist for artist, i in artist_dict.items()}

# targets
artists_num = [artist_dict[artist] for artist in artists]
targets = tf.data.Dataset.from_tensor_slices(artists_num)
target_vectorizer = layers.CategoryEncoding(num_tokens = len(artist_dict), output_mode="one_hot")
targets = targets.map(target_vectorizer)

# class weights for unbalanced datasets
balanced_class_weights = dict(enumerate(class_weight.compute_class_weight('balanced',
                                                 classes = list(artist_dict_rev.keys()),
                                                 y = artists_num)))
# num of examples from 1 artist can make the model biased for that if it dominates otherwise

# zipping
dataset = tf.data.Dataset.zip((inputs, targets))

# shuffling
n_examples = tf.data.experimental.cardinality(dataset).numpy()
dataset = dataset.shuffle(n_examples, seed = 42, reshuffle_each_iteration=False)

# !!!!! Important: setting the seed does not prevent different shuffling at each epoch, 
# !!!!! set reshuffle_each_iteration=False if used

# check dataset
for input_, target in dataset.take(3):
    print(input_)
    print(target)
    print(artist_dict_rev[tf.argmax(target).numpy()])
    print("\n")

print("Class counts and weights:")
print(Counter(artists_num))
print(balanced_class_weights)

"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# transforming to a tensorflow dataset\n\n# inputs\ninputs = tf.data.Dataset.from_tensor_slices(lines) #.repeat() #https://stackoverflow.com/questions/49531286/cannot-batch-tensors-with-different-shapes-in-component-0-with-tf-data-dataset\n\ninput_vectorizer = layers.TextVectorization(\n    standardize="lower_and_strip_punctuation",\n    split = "whitespace",\n    output_mode=\'int\',\n    max_tokens = 2000,\n    output_sequence_length = 10)\n\ninput_vectorizer.adapt(inputs)\ninputs = inputs.map(input_vectorizer)\n\n# target dict\nartist_dict = {artist: i for i, artist in enumerate(list(set(artists)))}\nartist_dict_rev = {i: artist for artist, i in artist_dict.items()}\n\n# targets\nartists_num = [artist_dict[artist] for artist in artists]\ntargets = tf.data.Dataset.from_tensor_slices(artists_num)\ntarget_vectorizer = layers.CategoryEncoding(num_tokens = len(artist_dict), output_mode="one_hot")\ntargets = targets.map(tar

In [29]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# splitting and batching

AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
val_size = 0.1

# splitting, batching
val_size = round(val_size*n_examples)
test_dataset = dataset.take(val_size).batch(batch_size=batch_size, drop_remainder=True)
val_dataset = dataset.skip(val_size).take(val_size).batch(batch_size=batch_size, drop_remainder=True)
train_dataset = dataset.skip(val_size*2).batch(batch_size=batch_size, drop_remainder=True)

# prefetching
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# splitting and batching\n\nAUTOTUNE = tf.data.AUTOTUNE\nbatch_size = 32\nval_size = 0.1\n\n# splitting, batching\nval_size = round(val_size*n_examples)\ntest_dataset = dataset.take(val_size).batch(batch_size=batch_size, drop_remainder=True)\nval_dataset = dataset.skip(val_size).take(val_size).batch(batch_size=batch_size, drop_remainder=True)\ntrain_dataset = dataset.skip(val_size*2).batch(batch_size=batch_size, drop_remainder=True)\n\n# prefetching\ntest_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)\nval_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)\ntrain_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)\n\nprint(len(train_dataset))\nprint(len(val_dataset))\nprint(len(test_dataset))\n'

In [30]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# defining our hyperparameter search space -> 2 methods: RealInterval, Discrete
hp_learning_rate = hp.HParam('learning_rate', hp.Discrete([0.1, 0.01, 0.001, 0.0001, 0.00001])) 
# currently possible with only two values for grid search
hp_optimizer = hp.HParam('optimizer', hp.Discrete(['adamw', 'sgd']))
hp_class_weights = hp.HParam('class_weights', hp.Discrete(['none', 'balanced']))
#hp_hidden_units = hp.HParam('hidden_units', hp.Discrete([8,32]))  # OR hp.Discrete([8, 32])

# fixed parameters and dimensions
params = {"vocab_size":input_vectorizer.vocabulary_size() +1,  # +1: Padding
            "embedding_dim": 256,
            "hidden_units": 128,
            "n_labels": len(artist_dict),
            "n_epochs": 10,
            "n_steps": 3*len(train_dataset)}


# initializing the logger
with tf.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
    hparams=[hp_learning_rate, hp_optimizer, hp_class_weights],#, hp_hidden_units],
    metrics=[hp.Metric("accuracy", display_name='Accuracy'),
            hp.Metric("precision", display_name= "Precision"),
            hp.Metric("recall", display_name = "Recall"),
            hp.Metric("f1", display_name = "F1")],
    )  # see notes for more info aout metrics

"""


'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# defining our hyperparameter search space -> 2 methods: RealInterval, Discrete\nhp_learning_rate = hp.HParam(\'learning_rate\', hp.Discrete([0.1, 0.01, 0.001, 0.0001, 0.00001])) \n# currently possible with only two values for grid search\nhp_optimizer = hp.HParam(\'optimizer\', hp.Discrete([\'adamw\', \'sgd\']))\nhp_class_weights = hp.HParam(\'class_weights\', hp.Discrete([\'none\', \'balanced\']))\n#hp_hidden_units = hp.HParam(\'hidden_units\', hp.Discrete([8,32]))  # OR hp.Discrete([8, 32])\n\n# fixed parameters and dimensions\nparams = {"vocab_size":input_vectorizer.vocabulary_size() +1,  # +1: Padding\n            "embedding_dim": 256,\n            "hidden_units": 128,\n            "n_labels": len(artist_dict),\n            "n_epochs": 10,\n            "n_steps": 3*len(train_dataset)}\n\n\n# initializing the logger\nwith tf.summary.create_file_writer(log_dir).as_default():\n    hp.hparams_config(\n    hparams=[hp_le

In [31]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# functions to run a trial with specified hyperparameters and fixed parameters

def hp_search_trial(log_dir, params, hparams):
    model = tf.keras.Sequential([
    layers.Embedding(params["vocab_size"], params["embedding_dim"], mask_zero = True), # masks zero paddings but not any other value
    #layers.Masking(mask_value = 0), -> NOT needed
    layers.GlobalAveragePooling1D(),
    layers.Dense(params["hidden_units"], activation = tf.nn.leaky_relu),
    # using Leaky ReLU: similar to ReLU with a little tweak for negative input values
    layers.Dense(params["n_labels"], activation = tf.nn.softmax)])

    if hparams[hp_optimizer] == "sgd":
        
        optimizer = tf.keras.optimizers.SGD(learning_rate=hparams[hp_learning_rate])
        
    elif hparams[hp_optimizer] == "adamw":  # updated version of adam
        
        optimizer = optimization.create_optimizer(init_lr=hparams[hp_learning_rate],
                                            num_train_steps=params["n_steps"],
                                            num_warmup_steps=round(0.1* params["n_steps"]),
                                            optimizer_type='adamw')

    class_weights = balanced_class_weights if hparams[hp_class_weights] == "balanced" else None

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )

    model.fit(train_dataset,validation_data=val_dataset, epochs=params["n_epochs"],
            callbacks=[tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                        histogram_freq=1, 
                                                        update_freq='batch')],
            class_weight = class_weights) 
    
    test_loss, test_accuracy = model.evaluate(test_dataset)
    
    true_labels = np.concatenate([y for x, y in test_dataset], axis=0).argmax(axis = -1)
    preds = model.predict(test_dataset).argmax(axis = -1)
    precision, recall, f1, support = precision_recall_fscore_support(true_labels, preds, average = "macro")
    # without avg. it will give out each class individually
    #precision, recall, f1, support = precision_recall_fscore_support(true_labels, preds)
    
    return test_accuracy, precision, recall, f1


def run(log_dir, params, hparams):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        
        test_accuracy, precision, recall, f1 = hp_search_trial(log_dir, params, hparams)
        tf.summary.scalar("accuracy", test_accuracy, step=1)
        # you can also log batch accuracy to see a trend, and train longer if needed
        tf.summary.scalar("precision", precision, step = 1)
        tf.summary.scalar("recall", recall, step = 1)
        tf.summary.scalar("f1", f1, step = 1)

        """

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# functions to run a trial with specified hyperparameters and fixed parameters\n\ndef hp_search_trial(log_dir, params, hparams):\n    model = tf.keras.Sequential([\n    layers.Embedding(params["vocab_size"], params["embedding_dim"], mask_zero = True), # masks zero paddings but not any other value\n    #layers.Masking(mask_value = 0), -> NOT needed\n    layers.GlobalAveragePooling1D(),\n    layers.Dense(params["hidden_units"], activation = tf.nn.leaky_relu),\n    # using Leaky ReLU: similar to ReLU with a little tweak for negative input values\n    layers.Dense(params["n_labels"], activation = tf.nn.softmax)])\n\n    if hparams[hp_optimizer] == "sgd":\n        \n        optimizer = tf.keras.optimizers.SGD(learning_rate=hparams[hp_learning_rate])\n        \n    elif hparams[hp_optimizer] == "adamw":  # updated version of adam\n        \n        optimizer = optimization.create_optimizer(init_lr=hparams[hp_learning_rate],\n 

## Running Grid Search

In [32]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

session_num = 0

for optimizer in hp_optimizer.domain.values:
    for learning_rate in hp_learning_rate.domain.values:#(hp_learning_rate.domain.min_value, hp_learning_rate.domain.max_value):
        for class_weights in hp_class_weights.domain.values:
            #for hidden_units in hp_class_weights.domain.values:  # (hp_hidden_units.domain.min_value, hp_hidden_units.domain.max_value):
                hparams = {
                  hp_optimizer: optimizer,
                  hp_learning_rate: learning_rate,
                  hp_class_weights: class_weights
                  }
                run_name = f"run{session_num}_{optimizer}_lr{learning_rate}_weights={class_weights}"
                print(f'--- Starting trial: {run_name}')
                print({h.name: hparams[h] for h in hparams})
                run(f'{log_dir}{run_name}', params, hparams)
                session_num += 1
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\nsession_num = 0\n\nfor optimizer in hp_optimizer.domain.values:\n    for learning_rate in hp_learning_rate.domain.values:#(hp_learning_rate.domain.min_value, hp_learning_rate.domain.max_value):\n        for class_weights in hp_class_weights.domain.values:\n            #for hidden_units in hp_class_weights.domain.values:  # (hp_hidden_units.domain.min_value, hp_hidden_units.domain.max_value):\n                hparams = {\n                  hp_optimizer: optimizer,\n                  hp_learning_rate: learning_rate,\n                  hp_class_weights: class_weights\n                  }\n                run_name = f"run{session_num}_{optimizer}_lr{learning_rate}_weights={class_weights}"\n                print(f\'--- Starting trial: {run_name}\')\n                print({h.name: hparams[h] for h in hparams})\n                run(f\'{log_dir}{run_name}\', params, hparams)\n                session_num += 1\n'

In [33]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

!tensorboard --logdir logs/hparam_tuning --port 9000
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n!tensorboard --logdir logs/hparam_tuning --port 9000\n'

In [34]:
"""# FROM: 11 Hyperparameter Search Tensorboard
# CHANGES: None

# if it is not working, try another port

# listing tensorboard instances
notebook.list()

# resetting tensorboard

# Linux
#!kill #ProcessID

# Windows
#!taskkill /f /pid #ProcessID
# you also have to delete .tensorflow.info in your %TEMP% directory
"""

'# FROM: 11 Hyperparameter Search Tensorboard\n# CHANGES: None\n\n# if it is not working, try another port\n\n# listing tensorboard instances\nnotebook.list()\n\n# resetting tensorboard\n\n# Linux\n#!kill #ProcessID\n\n# Windows\n#!taskkill /f /pid #ProcessID\n# you also have to delete .tensorflow.info in your %TEMP% directory\n'

# Saving the my_artists model


In [35]:
# All the above cells are run 6 times with the same hyperparameters that are obtained 
# using the pop_genre as context data (with most lines) to save time and have 
# comparable results of syntax and semantics

# from https://huggingface.co/docs/transformers/model_sharing

model_path = "/content/drive/MyDrive/EchoCanyon/saved_model/my_artists"



In [36]:
#https://huggingface.co/docs/transformers/model_sharing

model.save_pretrained(model_path)

In [37]:
model = TFAutoModelForCausalLM.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/EchoCanyon/saved_model/my_artists.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [38]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: None

prompt = "Sweet home"
n_generations = 5 # runs 

lyrics = prompt

for i in range(n_generations):
    
    lyrics += generator(lyrics[-100:], return_full_text = False)[0]["generated_text"]
    
print(lyrics)

#eos enf of sentence is used a  pad token

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Sweet home Chicago

(solo)

Six and three is a Mississippi queen
Come on, baba don't you want to go to go to go to go to go to go-all
Oh come on, baba don't you ain't you ain't you ain't you ain't you ain't got you
I ain't got you in the same old (I won't you)
And you're the blues, yes I go on behind
I need you're the whole night long


Well do the blues, my baby
Well
